# 🧠 AutoML Neural Network Search (Phase 2)

Welcome to the **AutoML Neural Network Architecture Search — Phase 2**.

This project builds a fully automated deep learning architecture search system designed to:

- 🔍 Automatically explore CNN + AutoEncoder structures.
- ⚙️ Handle flexible hyperparameter combinations.
- 🔢 Search across depth (layers) and width (filters, kernels, dense units, latent space).
- 🧮 Extract and print filters from trained convolutional layers.
- 🖼️ Visualize filters layer-wise (optional).
- 📊 Modular design: easy to extend for any image dataset.

---

## 🚀 Features

- **AutoML Search Engine**
  - Supports both `grid` and `random` search.
  - Random search uses internal clipping for faster execution.
  
- **Dynamic Model Builder**
  - Builds CNN AutoEncoders dynamically based on search parameters.

- **Filter Extraction**
  - Extracts weights (filters) from convolutional layers after training.

- **Filter Visualization**
  - Prints or plots filters to understand layer-wise feature extraction.

- **Completely Modular**
  - Simple file structure for easy extension and testing.

---

## ✅ Current Phase: Phase 2 PRO Version

- Multi-layer CNN AutoEncoders
- Fully parameterized search space
- Multiple convolutional layers handled automatically
- Clean filter extraction per layer
- Streamlined random search with sample clipping

---

## ⚠️ Note

> This project is intentionally simple, interpretable and fully transparent AutoML framework — perfect for:
> 
> - Learning
> - Experimentation
> - Extension to real-world datasets

---

## 🔧 Next Steps

- Build more advanced optimization (Phase 3)
- Add dataset auto-adaptation
- Implement scoring customization
- More efficient search strategies

---



In [ ]:
# AutoML_Demo_Template_V2.py

import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

from automl_search import AutoMLSearch
from cnn_autoencoder_builder_v2 import cnn_autoencoder_builder_v2
from get_plot_filters import plot_all_filters, list_conv_layers

# 1️⃣ Load and prepare dataset (replaceable with any dataset)
(x_train, _), (x_test, _) = mnist.load_data()
x = np.concatenate((x_train, x_test), axis=0)
x = x.astype('float32') / 255.0
x = np.reshape(x, (x.shape[0], 28, 28, 1))
X_train, X_val = train_test_split(x, test_size=0.2, random_state=42)

# Optional: subset to speed up testing
X_train, X_val = X_train[:5000], X_val[:1000]

# 2️⃣ Define parameter grid (fully flexible for your search space)
param_grid = {
    'num_conv_layers': [1, 2],
    'conv_filters_list': [[16], [16, 32]],
    'kernel_sizes_list': [[(3, 3)], [(3, 3), (3, 3)]],
    'pooling_type': ['max'],
    'dropout_rate': [0.2],
    'dense_units': [32],
    'activation': ['relu'],
    'latent_dim': [8]
}

# 3️⃣ Build and initialize AutoML search
search = AutoMLSearch(
    model_builder=cnn_autoencoder_builder_v2,
    param_grid=param_grid,
    mode='random',  # or 'grid'
    scoring='val_loss',
    n_jobs=1,
    epochs=5
)

# 4️⃣ Run the search
search.fit(X_train, X_val)

# 5️⃣ Show best parameters
print("\n✅ Best Hyperparameters Found:")
print(search.get_best_params())

# 6️⃣ Extract filters from best model
filters = search.extract_filters()

# 7️⃣ List available conv layers
list_conv_layers(search.get_best_model())

# 8️⃣ Plot filters for inspection
plot_all_filters(filters)

print("\n🎯 AutoML V2 PRO Demo Run Completed Successfully!")


In [ ]:
import itertools
import multiprocessing
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
import os

import itertools
import random
import numpy as np

class AutoMLSearch:

    def __init__(self, model_builder, param_grid, input_shape, mode='grid', n_jobs=1, epochs=10, scoring='val_loss'):
        self.model_builder = model_builder
        self.param_grid = param_grid
        self.input_shape = input_shape
        self.mode = mode
        self.n_jobs = n_jobs
        self.epochs = epochs
        self.scoring = scoring
        self.results = []

    def _param_combinations(self):
        keys = list(self.param_grid.keys())
        values = list(self.param_grid.values())

        # Convert nested lists to tuples for cartesian product
        processed_values = []
        for v in values:
            if isinstance(v[0], list): 
                processed_values.append([tuple(i) for i in v])
            else:
                processed_values.append(v)

        combos = list(itertools.product(*processed_values))

        for combo in combos:
            params = dict(zip(keys, combo))

            # Convert back to list where required
            if 'conv_filters_list' in params:
                params['conv_filters_list'] = list(params['conv_filters_list'])
            if 'kernel_sizes_list' in params:
                params['kernel_sizes_list'] = list(params['kernel_sizes_list'])
            yield params

    def fit(self, X_train, X_val):
        combinations = list(self._param_combinations())

        if self.mode == 'random':
            sample_size = min(20, len(combinations))  # Clip to 20 samples
            combinations = random.sample(combinations, sample_size)

        for params in combinations:
            try:
                model = self.model_builder(self.input_shape, params)
                history = model.fit(X_train, X_train, validation_data=(X_val, X_val), epochs=self.epochs, verbose=0)
                score = history.history[self.scoring][-1]
                self.results.append({'params': params, 'score': score, 'model': model})
                print(f"Params: {params} | Score: {score:.4f}")
            except Exception as e:
                print(f"Failed on params {params}: {e}")

        self.results.sort(key=lambda x: x['score'])
        self.best_model = self.results[0]['model'] if self.results else None
        self.best_params = self.results[0]['params'] if self.results else None

    def get_best_model(self):
        return self.best_model

    def get_best_params(self):
        return self.best_params

    def extract_filters(self):
        filters = []
        for layer in self.best_model.layers:
            if 'conv2d' in layer.name.lower():
                filters.append(layer.get_weights()[0])
        return filters



In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Flatten, Dense, Reshape

from tensorflow.keras import layers, models

def cnn_autoencoder_builder(input_shape, params):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    # Encoder
    for i in range(params['num_conv_layers']):
        filters = params['conv_filters_list'][i]
        kernel_size = params['kernel_sizes_list'][i]
        x = layers.Conv2D(filters, kernel_size, activation=params['activation'], padding='same')(x)

        if params['pooling_type'] == 'max':
            x = layers.MaxPooling2D(pool_size=(2,2))(x)
        elif params['pooling_type'] == 'avg':
            x = layers.AveragePooling2D(pool_size=(2,2))(x)

        if params['dropout_rate'] > 0:
            x = layers.Dropout(params['dropout_rate'])(x)

    x = layers.Flatten()(x)
    x = layers.Dense(params['dense_units'], activation=params['activation'])(x)
    encoded = layers.Dense(params['latent_dim'], activation=params['activation'])(x)

    # Decoder
    x = layers.Dense(params['dense_units'], activation=params['activation'])(encoded)
    x = layers.Dense(input_shape[0]*input_shape[1]*input_shape[2], activation='sigmoid')(x)
    decoded = layers.Reshape(input_shape)(x)

    model = models.Model(inputs, decoded)
    model.compile(optimizer='adam', loss='mse')
    return model



In [ ]:
# get_plot_filters.py

import matplotlib.pyplot as plt
import numpy as np
import math

def plot_layer_filters(model=None, filters=None, layer_name=None, layer_index=None):
    """
    Plots filters of a convolutional layer.

    Args:
        model: Trained Keras model (optional if filters provided).
        filters: Tuple of (filters, biases) directly from extract_filters().
        layer_name: Name of layer (if model provided).
        layer_index: Index of layer (if model provided).
    """
    # Determine how filters are provided
    if filters:
        filters, biases = filters
    elif model:
        if layer_name:
            layer = model.get_layer(name=layer_name)
        elif layer_index is not None:
            layer = model.layers[layer_index]
        else:
            raise ValueError("Provide either layer_name or layer_index")

        try:
            filters, biases = layer.get_weights()
        except:
            raise ValueError("Selected layer has no filters.")

    num_filters = filters.shape[-1]
    kernel_shape = filters.shape[:3]

    cols = 8
    rows = math.ceil(num_filters / cols)
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    axs = axs.flatten()

    for i in range(num_filters):
        f = filters[:, :, :, i]
        if f.shape[-1] == 1:
            f = f[:, :, 0]
        axs[i].imshow(f, cmap='viridis')
        axs[i].set_title(f'Filter {i+1}', fontsize=8)
        axs[i].axis('off')

    for j in range(i + 1, len(axs)):
        axs[j].axis('off')

    plt.suptitle(f"Filters | Shape: {kernel_shape} | Total Filters: {num_filters}", fontsize=12)
    plt.tight_layout()
    plt.show()


def list_conv_layers(model):
    """
    Utility to list all Conv2D layers in model.
    """
    print("Available Conv2D layers:")
    for idx, layer in enumerate(model.layers):
        if 'conv2d' in layer.name.lower():
            print(f"Index: {idx}, Name: {layer.name}, Filters: {layer.filters if hasattr(layer, 'filters') else 'N/A'}")


In [ ]:
# phase2_output_extractor.py

import numpy as np

def extract_and_print_filters(model):
    """
    Extracts and prints filter shapes and weights layer-wise for all Conv2D layers.

    Args:
        model: Trained Keras model.
    """
    print("\n Extracting Filters from Model...\n")
    
    for idx, layer in enumerate(model.layers):
        if 'conv2d' in layer.name.lower():
            try:
                filters, biases = layer.get_weights()
                shape = filters.shape  # (3, 3, in_channels, out_channels)
                
                print(f"Layer {idx} - {layer.name}")
                print(f"  Filter Shape: {shape}")
                
                # Print first filter as example
                print(f"\n  Filter 1 weights (kernel values):\n")
                print(np.round(filters[:, :, :, 0], 4))  # only print 1st filter

                print("-" * 50)
            except:
                print(f" No filters found in layer: {layer.name}")


In [ ]:
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
import numpy as np

from automl_search import AutoMLSearch
from cnn_autoencoder_builder import cnn_autoencoder_builder
from visualize_filters import print_filters
from get_plot_filters import plot_layer_filters, list_conv_layers


# Load data
(x_train, _), (x_test, _) = mnist.load_data()
x = np.concatenate((x_train, x_test), axis=0)
x = x.astype('float32') / 255.0
x = np.reshape(x, (x.shape[0], 28, 28, 1))
X_train, X_val = train_test_split(x, test_size=0.2, random_state=42)
X_train, X_val = X_train[:10000], X_val[:2000]

# Param grid
param_grid = {
    'num_conv_layers': [2, 3],
    'conv_filters_list': [
        [8],
        [8, 16],
        [8, 16, 32]
    ],
    'kernel_sizes_list': [
        [(3,3)],
        [(3,3), (3,3)],
        [(3,3), (3,3), (3,3)]
    ],
    'pooling_type': ['max', 'avg'],
    'dropout_rate': [0.1, 0.2, 0.3],
    'dense_units': [16, 32, 64],
    'activation': ['relu', 'tanh'],
    'latent_dim': [4, 8, 16]
}


# Search
search = AutoMLSearch(
    model_builder=cnn_autoencoder_builder,
    param_grid=param_grid,
    input_shape=(28,28,1),
    mode='random',
    epochs=3
)

search.fit(X_train, X_val)

# Best params
print("Best Hyperparameters:")
print(search.get_best_params())

# Filters
filters = search.extract_filters()
print("Filters")
print("*"*50)
print_filters(filters)
print("*"*50)
best_model = search.get_best_model()
# list_conv_layers(best_model)
from output_extractor import extract_and_print_filters
extract_and_print_filters(best_model)
plot_layer_filters(model=best_model, layer_index=1)

# from output_extractor import extract_and_print_filters
# extract_and_print_filters(best_model)

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from automl_search import AutoMLSearch
from cnn_autoencoder_builder import cnn_autoencoder_builder
from get_plot_filters import plot_layer_filters, list_conv_layers

import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split


(x_train, _), (x_test, _) = mnist.load_data()
x = np.concatenate((x_train, x_test), axis=0)
x = x.astype('float32') / 255.0
x = np.reshape(x, (x.shape[0], 28, 28, 1))
X_train, X_val = train_test_split(x, test_size=0.2, random_state=42)

param_grid = {
    'num_conv_layers': [2],
    'conv_filters_list': [[16, 32]],
    'kernel_sizes_list': [[(3,3), (3,3)]],
    'pooling_type': ['max'],
    'dropout_rate': [0.2],
    'dense_units': [32],
    'activation': ['relu'],
    'latent_dim': [8]
}

search = AutoMLSearch(
    model_builder=cnn_autoencoder_builder,
    param_grid=param_grid,
    input_shape=(28,28,1),
    epochs=1
)

search.fit(X_train, X_val)

assert search.get_best_model() is not None
assert search.get_best_params() is not None

filters = search.extract_filters()
assert len(filters) > 0

print("✅ All tests passed.")


In [ ]:
import numpy as np

import numpy as np

def print_filters(filters):
    for idx, f in enumerate(filters):
        print(f"\nLayer {idx+1} filters shape: {f.shape}")
        for i in range(f.shape[-1]):
            kernel = f[..., i]
            print(f"\nFilter {i+1} weights:\n{kernel}")

